## <center>Synchronous Computing</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 3620 </center>

#### <center> Synchronous Computation </center>

In a (fully) synchronous computation, all the processes synchronized at regular points, usually to exchange data or to making sure every process has gone through the same set of procedures (to update their own data) before proceeding.

In [7]:
%%writefile codes/mpi4py/nobarrier.py
#!/usr/bin/env python
# nobarrier.py
import time
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    time.sleep(5)
print ("process " + str(rank) + " is here")

Overwriting codes/mpi4py/nobarrier.py


In [8]:
!chmod 755 codes/mpi4py/nobarrier.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/nobarrier.py

process 1 is here
process 2 is here
process 3 is here
process 0 is here


In [12]:
%%writefile codes/mpi4py/barrier.py
#!/usr/bin/env python
# barrier.py
import time
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    time.sleep(5)
comm.Barrier()
print ("process " + str(rank) + " is here")

Overwriting codes/mpi4py/barrier.py


In [13]:
!chmod 755 codes/mpi4py/barrier.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/barrier.py

process 0 is here
process 1 is here
process 2 is here
process 3 is here


#### <center> Barrier </center>

- A basic mechanism for synchronizing processes - inserted at the point in each process where it must wait
- All processes can continue from this point when all the processes have reached it. 

Comm.Barrier()

Parameters:
- Comm (MPI comm) – communicator on which we are to block processes

<center> <img src="pictures/08/treebarrier1.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

<center> <img src="pictures/08/treebarrier2.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

<center> <img src="pictures/08/butterflybarrier1.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

#### <center> Prefix Sum Problem </center>

Given a list of numbers, $x_0, ..., x_{n-1}$, compute all partial summations, i.e:
- $x_0 + x_1$
- $x_0 + x_1 + x_2$
- $x_0 + x_1 + x_2 + x_3$
- $x_0 + x_1 + x_2 + x_3 + x_4$
- ...

Widely studied with practical applications in process allocation, data compaction, sorting, and polynomial evaluation. 

<center> <img src="pictures/08/prefixsum.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

In [14]:
%%writefile codes/mpi4py/prefixsum.py
#!/usr/bin/env python
# prefixsum.py
import numpy as np
import math
from mpi4py import MPI
comm = MPI.COMM_WORLD

rank = comm.Get_rank(); size = comm.Get_size(); N = 16

local_nums = np.zeros(int(N/size), dtype="int")
recv_sum = np.zeros(1, dtype="int")
local_sums = np.zeros(int(N/size), dtype="int")

for i in range(0,int(N/size)):
    local_nums[i] = rank * int(N/size) + i
    local_sums[i] += np.sum(local_nums[0:(i+1)])

print("Process ", rank, " has ", local_nums)
print("Process ", rank, " has ", local_sums)

for i in range(0, int(math.log2(size))):
    distance = int(math.pow(2,i))
    if (rank == 0):
        print (distance)
    if (rank < (size - distance)):
        comm.Send(local_sums[int(N/size) - 1], dest = rank + distance, tag = 0)
#        print ("Process ", rank, " sends to ", rank + distance)
    if (rank >= distance):
        status = MPI.Status()
        comm.Recv(recv_sum, source = rank - distance, tag = 0, status = status);
#        print ("Process ", rank, " receives from ", rank - distance, " values ", recv_nums)
        for j in range(0,int(N/size)):
            local_sums[j] += recv_sum[0]
    print("Process ", rank, " has ", local_sums)

Writing codes/mpi4py/prefixsum.py


In [15]:
!chmod 755 codes/mpi4py/prefixsum.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/prefixsum.py

Process  0  has  [0 1 2 3]
Process  0  has  [0 1 3 6]
1
Process  0  has  [0 1 3 6]
2
Process  1  has  [4 5 6 7]
Process  1  has  [ 4  9 15 22]
Process  2  has  [ 8  9 10 11]
Process  2  has  [ 8 17 27 38]
Process  2  has  [30 39 49 60]
Process  3  has  [12 13 14 15]
Process  3  has  [12 25 39 54]
Process  3  has  [50 63 77 92]
Process  1  has  [10 15 21 28]
Process  1  has  [10 15 21 28]
Process  0  has  [0 1 3 6]
Process  3  has  [ 78  91 105 120]
Process  2  has  [36 45 55 66]
